In [11]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("C:\Users\shrey\Downloads\SabudhProject\shreya.csv")
df.head()
df.info()

In [13]:
df.isna().sum()

Unnamed: 0                  0
isbn                        0
text_reviews_count          0
series                      0
country_code                0
language_code               0
popular_shelves             0
is_ebook                    0
average_rating              0
similar_books               0
description                 0
format                  10515
link                        0
authors                     0
publisher                   0
num_pages                   0
publication_day         23664
isbn13                   4123
publication_month           0
publication_year            0
url                         0
image_url                   0
book_id                     0
ratings_count               0
work_id                     0
title                       1
title_without_series        1
dtype: int64

In [14]:
df.drop_duplicates(subset='isbn', keep='first', inplace=True)
df.drop_duplicates(subset='title', keep='first', inplace=True)

In [15]:
data = df['title'].value_counts() > 1
df2 = pd.DataFrame(list(data.values))
df2.value_counts()

0    
False    113648
Name: count, dtype: int64

In [16]:
df.shape

(113649, 27)

In [17]:
import random
#Creating a synthetic user dataset
num_users = 3000

user_book_list = []

for user in range(1, num_users + 1):
    num_books = random.randint(30, 35)
    sampled_books = df.sample(num_books)
    for _, row in sampled_books.iterrows():
        user_book_list.append({
            "user_id": user,
            "isbn": row["isbn"],
            "title": row["title_without_series"],
            "average_rating": row['average_rating'],
            "description": row['description']
        })
user_book_df = pd.DataFrame(user_book_list)

print(user_book_df.head())


   user_id        isbn                                              title  \
0        1  0578169185                                         Every Love   
1        1  0451233441  The Unbearable Lightness of Dragons (Light Dra...   
2        1  6066860178                           Scenariu pentru dragoste   
3        1  1507885806            Yummy Addictions (The Yummy Trilogy #1)   
4        1  1503953491                             Mi segunda primera vez   

   average_rating                                        description  
0            4.13  Post - Living in the after, the events that to...  
1            4.08  Ysolde Bouchier is still coming to terms with ...  
2            3.98  A woman dares to defy society.\nA man resists ...  
3            3.79  Valentina "Vale" Zamora is a twenty-eight-year...  
4            3.80  Finalist for the 2013 RITA(R) Award for Best F...  


In [18]:
user_book_df

,user_id,isbn,title,average_rating,description
0,1,0578169185,Every Love,4.13,"Post - Living in the after, the events that to..."
1,1,0451233441,The Unbearable Lightness of Dragons (Light Dra...,4.08,Ysolde Bouchier is still coming to terms with ...
2,1,6066860178,Scenariu pentru dragoste,3.98,A woman dares to defy society.\nA man resists ...
3,1,1507885806,Yummy Addictions (The Yummy Trilogy #1),3.79,"Valentina ""Vale"" Zamora is a twenty-eight-year..."
4,1,1503953491,Mi segunda primera vez,3.80,Finalist for the 2013 RITA(R) Award for Best F...
...,...,...,...,...,...
97409,3000,0751539562,The Summer Of Secrets,3.87,Hope Gardener has always been a drifter. A car...
97410,3000,0374281351,Night Wraps the Sky: Writings by and about May...,4.22,"From the time his first, futurist poems were p..."
97411,3000,147500382x,"Leopard Moon (Moon, #1)",3.77,How do you disappear when you come from a fami...
97412,3000,9944825672,"Kalpsiz (The House of Rohan, #1)",3.81,Passion Always Wins\nVery few people can witne...


In [14]:
user_book_df.to_csv('user_book_dataset2.csv')

In [40]:
import random
# shuffle user ID's
users = [x for x in range(1, num_users + 1)]
random.shuffle(users)

# extract 90% of customer ID's
users_train = [users[i] for i in range(round(0.9*len(users)))]
users_val = [users[i] for i in range(len(users)) if users[i] not in users_train]

# split data into train and validation set
train_df = user_book_df[user_book_df['user_id'].isin(users_train)]
validation_df = user_book_df[~user_book_df['user_id'].isin(users_train)]

In [21]:
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
import warnings;
warnings.filterwarnings('ignore')

In [41]:
# list to capture book history of the users
books_train = []

for user in users_train:
    temp = train_df[train_df["user_id"] == user]["description"].tolist()
    books_train.append(temp)

In [43]:
# list to capture book history of the users(validation set)
books_val = []

# populate the list with the product codes
for user in users_val:
    temp = validation_df[validation_df["user_id"] == user]["description"].tolist()
    books_val.append(temp)

In [44]:
# train word2vec model
model = Word2Vec(sentences=books_train, window=5, sg=1, min_count=1, workers=4)

In [45]:
def recommend_items(item_id, top_n=5):
    if item_id in model.wv:
        return model.wv.most_similar(item_id, topn=top_n)
    else:
        return []
        
def user_embedding(user_items):
    vectors = [model.wv[item] for item in user_items if item in model.wv]
    return np.mean(vectors, axis=0) if vectors else None

def recommend_for_user(user_items, top_n=5):
    user_vec = user_embedding(user_items)
    if user_vec is None:
        return []
    return model.wv.similar_by_vector(user_vec, topn=top_n)

In [46]:
import re

def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    idx = sentence.find('title dtype object')
    sentence = sentence[:idx]
    sentence.strip()

    return sentence

recommendations = []
for i in range(len(valis)):
    book_history = books_val[i]
    user_id = valis[i]
    rec = recommend_for_user(book_history, 50)

    j = 0
    titles = []
    while len(titles) < 5 and j < 50:
        title = df[df['description'] == rec[j][0]]['title'].astype(str)
        title = preprocess_text(str(title))
        if title not in titles:
            titles.append(title)
        j +=1  
        
    recommendations.append({
        "user_id": user_id,
        "titles": titles
    })

val_rec_df = pd.DataFrame(recommendations)

In [57]:
val_rec_df.loc[0]['titles']

[' Must Love Breeches Name ',
 ' The Prodigal Daughter Name ',
 ' Have Touched You Name ',
 ' In the Arms of Marquess Rogues of the Sea Name ',
 ' The Argonautika The Story of Jason and the Qu Name ']

In [58]:
val_rec_df

,user_id,titles
0,1330,"[ Must Love Breeches Name , The Prodigal Daug..."
1,507,"[ Dangerous as Sin Bligh Family Name , The Ma..."
2,876,"[ Another Mother Life Name , Rake with Frozen..."
3,1377,"[ In Love with the Bronc Rider Name , r Lydia..."
4,543,"[ Damaso Claims His Heir Name , The Storm Tha..."
...,...,...
295,1255,"[ Kept The Enforcers Name , The Partition of ..."
296,520,"[ Magia proibita Name , Les Destinees Name , ..."
297,2141,"[ The Rhythm of Your Heartbeat Name , Star Ra..."
298,2007,[ Never Courted Suddenly Wed Scandalous Season...
